In [1]:
import numpy as np
import matplotlib 
matplotlib.use('nbagg')
import matplotlib.pyplot as plt
import random

In [111]:
# Function to generate the Dataset for a known hypothesis
# Assumptions: Range of values on the x and y axis are the same
#              Code is only for classifying data in 2D
def gen_data_perceptron(llim,ulim,target_weight_3D,N):
    
    """
    llim: a scalar defining the lower limit on the x and y axis
    ulim: a scalar defining the upper limit on the x and y axis
    target_weight_3D: a column vector of 3x1 listing the known weights used for generating the target hypothesis
    N: Total number of data points
    Output: Returns a tuple with following elements at respective indices-
        index        element
        0 -          a numpy array of Nx4 with the first column being all 1s, second and third column 
                     consists of N points generated uniformly between llim and ulim
    """
    x1_sample = np.append(np.ones((N,1)),np.random.uniform(llim,ulim,N).reshape(N,1),axis=1)
    x2_sample = np.random.uniform(llim,ulim,N).reshape(N,1)
    data_partial = np.append(x1_sample,x2_sample,axis=1)    
    target = np.dot(data_partial,target_weight_3D)
    mask = target>0
    category_values = np.where(mask,np.ones((N,1)),-1)
    data_complete = np.append(data_partial,category_values,axis=1)
    
    return data_complete

# Function to evaluate the pointwise gradient of the cross entropy error function
def grad_eval(weight_current,x_vec,category_val):
    """
    Inputs
    weight_init: current set of weights (a column vector)
    x_vec: input point (a 3x1 column vector)
    category_val: +1 or -1 depending upon the label of the point
    
    Output:
    Returns the gradient evaluated at the current set of weights(a column vector)
    """
    gradient = -x_vec*category_val/(1 + np.exp(category_val*np.dot(weight_current.reshape(3,),x_vec.reshape(3,))))
    return gradient

# SGD update for Logistic regression for cross entropy error function
def SGD_logistic_regression(weights_init,data_complete,stopping_threshold,eta):
    """
    Inputs
    weight_init: current set of weights (a column vector)
    x_vec: input point (a 3x1 column vector)
    category_val: +1 or -1 depending upon the label of the point
    
    Output:
    Returns a tuple with following elements at the respective indices:
    0 - number of epochs
    1 - final set of weights (a 3x1 column vector) 
    """
    no_of_epochs = 0
    weight_diff = 1
    while weight_diff>=stopping_threshold:
        weights_old = weights_init + np.zeros((3,1))
        order_vec = np.random.permutation(np.arange(data_complete.shape[0]))
        for i in order_vec:
            x_vec = data_complete[i,0:3].reshape(-1,1)
            category_val = data_complete[i,3]
            weights_init = weights_init - (eta*grad_eval(weights_init,x_vec,category_val))
        diff_vec = (weights_init - weights_old).reshape(3,)
        weight_diff = (np.dot(diff_vec,diff_vec))**0.5
        no_of_epochs = no_of_epochs + 1
    weights_updated = weights_init
    #print("The number of epochs it took to converge: ", no_of_epochs)
    #print("The final set of weights are : ", weights_updated)
    #print("The magnituded of weight difference is : ",weight_diff)
    return(no_of_epochs,weights_updated)

In [129]:
# Defining the parameters for parameter estimation for Logistic regression model and running SGD
llim = -1
ulim = 1
target_weights_3D = np.random.uniform(llim,ulim,3).reshape(-1,1)
N = 100
weights_init = np.zeros((3,1))
stopping_threshold = 0.01
eta = 0.01
data_complete = gen_data_perceptron(llim,ulim,target_weights_3D,N)
out_res = SGD_logistic_regression(weights_init,data_complete,stopping_threshold,eta)

array([[ 5.77694911],
       [-3.32204369],
       [-7.91212046]])

In [92]:
# Computing Eout
iters_test = 100
Eout = 0
for i in range(iters_test):
    # Training part
    target_weights_3D = np.random.uniform(llim,ulim,3).reshape(-1,1)
    weights_init = np.zeros((3,1))
    data_complete = gen_data_perceptron(llim,ulim,target_weights_3D,N)
    out_res = SGD_logistic_regression(weights_init,data_complete,stopping_threshold,eta)
    
    # Testing part
    data_test = gen_data_perceptron(llim,ulim,target_weights_3D,N)
    for i in range(N):
        Eout = Eout + np.log(1 + np.exp(-data_test[i,3]*np.dot(out_res[1].reshape(3,),data_test[i,0:3])))
print("The average out of sample error is: ", Eout/(N*iters_test))

The average out of sample error is:  0.0848111083495
